# Feature Selection 
This file performs feature selection for SVC, logistic regression, AdaBoost, and the Bayes Classifier to PA classification. The feature selection is done by first creating a power set using all possible features, then training the models on the combinations of features and then storing the scores. This notebook is a helper notebook for CMM-model-assessment.ipynb.

## Load and Clean Data
First we import the necessary packages.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Next, we load and clean the data. Then we merge the 3 dataframes into one and then split the resulting dataframe into two. 
In the first group, we have the data corresponding to pharmacy fills in which a PA was not requested and in the second we have the data corresponding to prescriptions for which a PA was requested.  

In [ ]:
# Load Data
df_date=pd.read_csv("data/dim_date.csv")
df_claim=pd.read_csv("data/dim_claims.csv")
df_pa=pd.read_csv("data/dim_pa.csv")
df_bridge=pd.read_csv("data/bridge.csv")

# Clean data so all reject_code values are integers
df_claim['reject_code'] = df_claim.reject_code.fillna(0).astype(int)

# Merge the data frames
df_main = pd.merge(df_claim, df_bridge, on='dim_claim_id')
df_main = pd.merge(df_main, df_pa, how='left', on='dim_pa_id')
df_main = pd.merge(df_main, df_date, how='left', on='dim_date_id')

# split the data frames into two -- PA requested or not
df_main_wPA = df_main[~np.isnan(df_main.pa_approved)].copy()
df_main_noPA = df_main[np.isnan(df_main.pa_approved)].copy()

Here is the dataframe corresponding to pharmacy fills with no PA requested. 

In [ ]:
df_main_noPA.head()

Here is the data frame corresponding to pharmacy fills with a PA requested.

In [ ]:
df_main_wPA.head()

Now we use one-hot encoding to turn the categorical features into binary features. There are two categorical features; the bin i.e. the payer and the drug type.

In [ ]:
# encode all categorical features in the case that a PA was requested 
df_aug=df_main_wPA.copy()
df_aug['70'] = pd.get_dummies(df_aug['reject_code'])[70]
df_aug['75'] = pd.get_dummies(df_aug['reject_code'])[75]
df_aug['76'] = pd.get_dummies(df_aug['reject_code'])[76]
df_aug['bin417380']=pd.get_dummies(df_aug['bin'])[417380]
df_aug['bin999001']=pd.get_dummies(df_aug['bin'])[999001]
df_aug['bin417740']=pd.get_dummies(df_aug['bin'])[417740]
df_aug['bin417614']=pd.get_dummies(df_aug['bin'])[417614]
df_aug['drug_A']=pd.get_dummies(df_aug['drug'])['A']
df_aug['drug_B']=pd.get_dummies(df_aug['drug'])['B']
df_aug['drug_C']=pd.get_dummies(df_aug['drug'])['C']

Now we make the train-test split.

In [ ]:
from sklearn.model_selection import train_test_split

# keep all except temporal features for now 
X=df_aug[['70', '75', '76', 'bin417380', 'bin999001','bin417740', 'bin417614','correct_diagnosis', 'contraindication', 'tried_and_failed', 'drug_A', 'drug_B','drug_C']]
y=df_aug[['pa_approved']]
X_train_gen,X_test_gen,y_train_gen,y_test_gen = train_test_split(X,y,
                                                test_size=.2,
                                                shuffle=True,
                                                stratify=y)

## Models of Interest
We will analyze the following classifiers: 
1. logistic regression;
2. Bayes classifier;
3. random forests;
4. linear support vector machine; 
5. AdaBoost. 

We first import the models and load the class for the Bayes/Group Average classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone

In [ ]:
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

class GroupAverageClassifier(BaseEstimator, TransformerMixin, ClassifierMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.approved_count_ = Counter()
        self.total_count_ = Counter()
        for r, t in zip(X, y):
            g = tuple(r)
            self.approved_count_[g] += t
            self.total_count_[g] += 1
        return self
    
    def transform(self, X, y=None):
        return self.predict_proba(X)
    
    def predict(self, X):
        prob = self.predict_prob(X)
        pred = (prob > 0.5).astype(int)
        return pred
    
    def predict_prob(self, X):
        prob = np.zeros(X.shape[0])
        eps = 1e-4
        for i, r in enumerate(X):
            g = tuple(r)
            prob[i] = self.approved_count_[g] / (self.total_count_[g] + eps)
        return prob

## Metrics of Interest 
We will be interested in accuracy, recall, precision, f1 and the area under the roc curve. 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, balanced_accuracy_score

In [ ]:
def get_func_name(f):
    name = f.__name__
    if name.endswith('_score'):
        name = name[:-6]
    return name

default_scores = [accuracy_score, precision_score, recall_score,f1_score, roc_auc_score, balanced_accuracy_score]
default_names = [get_func_name(f) for f in default_scores]

We now write the functions that build the powerset from all of the features and evaluate the models on this collection of features. 

In [ ]:
def powerset_no_empty(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
            
    return power_set[1:]

def get_all_scores(model):    
    cutoff = .5
    acc = np.empty(len(possible_features))
    precision = np.empty(len(possible_features))
    recall = np.empty(len(possible_features))
    f1 = np.empty(len(possible_features))
    roc = np.empty(len(possible_features))
    acc_t = np.empty(len(possible_features))
    precision_t = np.empty(len(possible_features))
    recall_t = np.empty(len(possible_features))
    f1_t = np.empty(len(possible_features))
    roc_t=np.empty(len(possible_features))

    df=pd.DataFrame(columns=['acc', 'prec', 'recall', 'f1', 'roc', 'acc_t','prec_t', 'recall_t', 'f1_t', 'roc_t' ])

    for j in range(len(possible_features)):
        ## get X and y
        print(j)
        X_mini = np.array(X_train_gen[possible_features[j]])

        y_train=np.array(y_train_gen).ravel()

        # Cloning the regression makes a fresh regression 
        # model for each run
        clone_reg = clone(model)

        # fit the model
        clone_reg.fit(X_mini, y_train)
        ## assign the value based on the cutoff
        y_train_pred = clone_reg.predict(X_mini)

        X_mini_t = np.array(X_test_gen[possible_features[j]])
        y_test=np.array(y_test_gen).ravel()
        
        y_test_pred = clone_reg.predict(X_mini_t)


        acc[j] = accuracy_score(y_train_pred,y_train)
        precision[j] = precision_score(y_train,y_train_pred)
        recall[j] = recall_score(y_train,y_train_pred)
        f1[j] = f1_score(y_train,y_train_pred)
        roc[j]=roc_auc_score(y_train, y_train_pred)

        acc_t[j] = accuracy_score(y_test_pred,y_test)
        precision_t[j] = precision_score(y_test,y_test_pred)
        recall_t[j] = recall_score(y_test,y_test_pred)
        f1_t[j] = f1_score(y_test,y_test_pred)
        roc_t[j]=roc_auc_score(y_test, y_test_pred)

    df['acc']=acc
    df['acc_t']=acc_t
    df['prec']=precision
    df['prec_t']=precision_t
    df['recall']=recall
    df['recall_t']=recall_t
    df['f1']=f1
    df['f1_t']=f1_t
    df['roc']=roc
    df['roc']=roc_t

    return df


The features we are interested in are given below. Recall that when we encode a categorical feature with $K$ classes via one-hot encoding, we only keet $K-1$ of the resulting binary features. 

In [ ]:
possible_features=powerset_no_empty(['70', '75', 'bin417380','bin417740', 'bin417614','correct_diagnosis', 'contraindication', 'tried_and_failed', 'drug_A', 'drug_B'])

### Logistic Regression

In [ ]:
log_reg=LogisticRegression()
df= get_all_scores(log_reg)

In [ ]:
df.sort_values(by='roc', ascending=False).head()

Our goal is to extract the features with the best roc score. However, the baseline model (see pa_classifier.ipynb) has an accuracy of .73, so we reject the first two feature combinations here. 

In [ ]:
print('We will use features: ', possible_features[656], 'for logistic regression.')

### Linear Support Vector Machine

In [ ]:
SVC = LinearSVC(C=1)
df_SVC=get_all_scores(SVC)

In [ ]:
df_SVC.sort_values(by='roc', ascending=False).head()

Again, we reject the first two feature combinations because the accuracy is too low and we choose the next combination.

In [ ]:
print('We will use features: ',possible_features[658],'for the linear support vector machine.')

### AdaBoost

In [ ]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
            n_estimators = 10,
            algorithm="SAMME.R",
            learning_rate = 0.5
        )
df_ada=get_all_scores(ada_clf)

In [ ]:
df_ada.sort_values(by='roc', ascending=False).head()

This time, we reject the first three feature combinations for hacing accuracy scores that are too low. 

In [ ]:
print('We will use features:',possible_features[838],'for AdaBoost.')

### Bayes Classifier

In [ ]:
bayes_clf=GroupAverageClassifier()
df_bayes=get_all_scores(bayes_clf)

In [ ]:
df_bayes.sort_values(by='roc', ascending=False).head()

We reject the first two feature combinations and choose the third. 

In [ ]:
print('We will use features: ', possible_features[669],'for the Bayes Classifier.')